In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

#
# data
#
from techminer import *


# Records(directory="../data/processed/debug").import_records(source="../data/tmp/debug-scopus.csv", filetype="scopus")
# Records(directory="../data/processed/debug").process_records()
# Records(directory="../data/processed/debug").coverage()
# Records(directory="../data/processed/debug").describe()
# Records(directory="../data/processed/debug").core_sources()
# Records(directory="../data/processed/debug").core_authors()

# wos_df = load_file(filepath="techminer/tests/data/savedrecs.txt", filetype="wos", datastorepath="./datastore")
# dim_df = load_file(filepath="techminer/tests/data/dimensions.csv", filetype="dimensions", datastorepath="./datastore")

In [2]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

# TermAnalyzer(directory_or_records="../data/processed/debug", column='authors', top_by='num_records').treemap('num_records')
Worldmap(directory_or_records="../data/processed/debug").plot()


,num_records,global_citations,local_citations
united states,125,5432,0
germany,67,319,0
italy,47,138,0
united kingdom,43,272,0
canada,27,62,0
...,...,...,...
monaco,1,2,0
mauritius,1,0,0
macedonia,1,1,0
latvia,1,10,0


In [ ]:





# apply_thesaurus(directory_or_records="../data/processed/debug", thesaurus_filepath="../data/processed/debug/keywords.txt", input_column="author_keywords", output_column="author_keywords_thesaurus")
# association_index(co_occurrence_matrix(directory_or_records="../data/processed/debug", column="author_keywords"), association='jaccard')
# bradford_plot(directory_or_records="../data/processed/debug")
# co_occurrence_matrix(directory_or_records="../data/processed/debug", column="author_keywords", by='author_keywords')
# co_occurrence_matrix(directory_or_records="../data/processed/debug", column="author_keywords", by='countries')
# core_authors(directory_or_records="../data/processed/debug")
# core_sources(directory_or_records="../data/processed/debug")
# coverage(directory_or_records="../data/processed/debug")
# create_thesaurus(directory_or_records="../data/processed/debug", column="keywords", thesaurus_filepath="../data/processed/debug/test_thesaurus.txt")
# impact_analysis(directory_or_records="../data/processed/debug", column="authors")
# lotka_plot(directory_or_records="../data/processed/debug")
# most_cited_documents(directory_or_records="../data/processed/debug")
# summary(directory_or_records="../data/processed/debug")
# tf_matrix(directory_or_records="../data/processed/debug", column="author_keywords")
# tfidf_matrix(tf_matrix(directory_or_records="../data/processed/debug", column="author_keywords"))
# worldmap(count_documents_by_term(directory_or_records="../data/processed/debug", column="countries"))
# collaboration_analysis(directory_or_records="../data/processed/debug", column="authors")

In [ ]:
count_documents_by_term(directory_or_records="../data/processed/debug", column="country")

In [ ]:
import nltk
nltk.__version__

In [ ]:
#
# query module
#
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.query import * 


# count_documents_by_term(directory_or_records="../data/processed/debug", column='authors', sep='; ')
# count_documents_by_year(directory_or_records="../data/processed/debug")
# count_global_citations_by_term(directory_or_records="../data/processed/debug", column='authors', sep='; ')
# count_global_citations_by_year(directory_or_records="../data/processed/debug")
# count_local_citations_by_term(directory_or_records="../data/processed/debug", column='authors', sep='; ')
# count_local_citations_by_year(directory_or_records="../data/processed/debug")
# count_terms_by_column(directory_or_records="../data/processed/debug", sep="; ", column='authors')
# mean_global_citations_by_year(directory_or_records="../data/processed/debug")
# mean_local_citations_by_year(directory_or_records="../data/processed/debug")
# term_analysis(directory_or_records="../data/processed/debug", column='authors')
# time_analysis(directory_or_records="../data/processed/debug")

# termino por año <----

# most_cited_documents(directory_or_records="../data/processed/debug")
# compute_impact_index(directory_or_records="../data/processed/debug", column='authors')




# ---
# worldmap(count_documents_by_term(directory_or_records="../data/processed/debug", column='countries'))


In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.data import *

# coverage(directory_or_records="../data/processed/debug")
describe_records(directory_or_records="../data/processed/debug")

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

#
# features
#
from techminer.features import *

# extract_coutries(directory_or_records="../data/processed/debug", column='affiliations')






In [ ]:
#
# query modules
#
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.query import * 
from techminer.visualization import worldmap



# term_analysis(directory_or_records="../data/processed/debug", column='authors')
### count_terms_by_column(directory_or_records="../data/processed/debug", sep="; ", column='authors')
# time_analysis(directory_or_records="../data/processed/debug")
### count_local_citations_by_year(directory_or_records="../data/processed/debug")
### count_global_citations_by_year(directory_or_records="../data/processed/debug")
### count_documents_by_year(directory_or_records="../data/processed/debug")
### mean_global_citations_by_year(directory_or_records="../data/processed/debug")
# most_cited_documents(directory_or_records="../data/processed/debug")
# compute_impact_index(directory_or_records="../data/processed/debug", column='authors')




# ---
# worldmap(count_documents_by_term(directory_or_records="../data/processed/debug", column='countries'))



In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import Records

Records(directory)

# core_sources(directory_or_records="../data/processed/debug")
# core_authors(directory_or_records="../data/processed/debug")
# lotka(core_authors(directory_or_records="../data/processed/debug"), colormap='Blues', figsize=(6,3))


In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


from techminer.visualization import bradford

bradford(directory_or_records="../data/processed/debug")

In [ ]:
import pandas as pd

from techminer.data.records import load_records

# sorted(load_records(directory="../data/processed/debug").columns)
load_records(directory="../data/processed/debug")

In [ ]:
from src.features.apply_keywords_thesaurus import apply_keywords_thesaurus
from src.features.apply_institutions_thesaurus import apply_institutions_thesaurus

# Ok!
apply_keywords_thesaurus(datastoredir="../data/processed/debug")
apply_institutions_thesaurus(datastoredir="../data/processed/debug")

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.analysis import Bradford

bradford = Bradford(datastoredir="../data/processed/debug")

bradford.plot(max_items=8, cmap='Blues', figsize=(8, 5), fontsize=9)

In [ ]:
bradford.sources_

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from src.analysis import TopDocuments

TopDocuments(datastoredir="../data/processed/debug").detailed_

In [ ]:
from techminer.analysis.top_documents import TopDocuments

TopDocuments(datastorepath="./datastore").print()

In [ ]:
from techminer.analysis.core_authors import CoreAuthors

CoreAuthors(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.core_authors import CoreAuthors

CoreAuthors(datastorepath="./datastore").plot(figsize=(8, 4))

In [ ]:
from techminer.analysis.core_sources import CoreSources

CoreSources(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.coverage import Coverage

Coverage(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.worldmap import WorldMap

WorldMap(datastorepath="./datastore").plot(figsize=(9, 5), colormap='Blues')

In [ ]:
import pandas as pd

pd.read_csv("../data/processed/debug/records.csv").author_keywords_thesaurus